In [16]:
import scrapy    #Utilizar en un entorno como Visual Studio Code

In [22]:
class Spider12(scrapy.Spider):
    name = 'Spider12'
    allowed_domains = ['pagina12.com.ar']
    custom_settings = {'DEPTH_LIMIT': 2,
                       'FEED_FORMAT': 'json',
                       'FEED_URI': 'resultados.json',
                       'FEED_EXPORT_ENCODING': 'utf-8',}
                       
    start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/ciencia',
                  'https://www.pagina12.com.ar/secciones/el-mundo',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/contratapa']
   
    def parse(self, response):
        # Artículo promocionado
        nota_promocionada = response.xpath('//div[@class="featured-article__container"]//h2/a/@href').get()
        if nota_promocionada is not None:
            yield response.follow(nota_promocionada, callback=self.parse_nota)

        # Listado de notas
        notas = response.xpath('//ul[@class="article-list"]//li//a/@href').getall()
        for nota in notas:
            yield response.follow(nota, callback=self.parse_nota)

        # Link a próxima página
        next = response.xpath('//a[@class="pagination-btn-next"]/@href').get()
        if next is not None:
            yield response.follow(next, callback=self.parse)

    def parse_nota(self, response):
        # Parseo la nota
        titulo = response.xpath('//h1[@class="article-title"]/text()').get()
        cuerpo = ''.join(response.xpath('//div[@class="article-text"]/p/text()').getall())
        yield {'url': response.url,
               'titulo': titulo,
               'cuerpo': cuerpo}


In [23]:
from scrapy.crawler import CrawlerProcess

In [24]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

2020-04-16 13:41:35 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-16 13:41:35 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-04-16 13:41:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-16 13:41:35 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 2,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'FEED_FORMAT': 'json',
 'FEED_URI': 'resultados.json'}
2020-04-16 13:41:35 [scrapy.extensions.telnet] INFO: Telnet Password: 2730606f55759609
2020-04-16 13:41:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogS

ReactorNotRestartable: 